<a href="https://colab.research.google.com/github/gbessardon/Create_plots/blob/main/Open_UW_get_projection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries

In [ ]:
!pip install rasterio

In [ ]:
from osgeo import gdal
import matplotlib.image as img
import os
import numpy as np
import tifffile as tif
import xml.etree.ElementTree as ET
import tifffile as tif
import matplotlib.pyplot as plt
import rasterio
import cv2
from rasterio import plot
from rasterio.plot import show
from skimage import exposure

# Link to the folder
https://drive.google.com/drive/folders/1tqy81Szl9owtgwFUSkdpvke_aGKS9jF-?usp=sharing
The UW map is divided by tiles which are for ireland projected in UTM29 and UTM30
the folder is structure is:
/content/drive/MyDrive/UW_map/:


*   T29

> Dublin

> ...




*  T30


> Sea Dublin


> Sea Atrim



> Down







In [ ]:
dirfiles='/content/drive/MyDrive/UW_map/T29/Dublin'
primaryorsecondary=1 #1 Primary 2 secondary

In [ ]:
def directorystructure(dirfiles,primaryorsecondary):
  if 'T29' in dirfiles:
    projection='EPSG:32629' # UTM 29 code
  if 'T30' in dirfiles:
    projection='EPSG:32630' # UTM 30 code

  if primaryorsecondary==1:
    pathending='Primary Prediction.tif'
  if primaryorsecondary==2:
    pathending='Secondary Prediction.tif'

  predictionfn=os.path.join(dirfiles,pathending)
  
  return(projection,predictionfn)

# Declare functions

## Functions to get the metadata

In [ ]:
def sentinel2safepath(path):
  """
  obtain the safe path to the sentinel-2 .SAFE dir containing image and it's metadata
  path=the city path i.e /content/drive/MyDrive/UW_map/T29/Dublin
  return the sentinel-2 .SAFE dir containing image and it's metadata
  """
  safepath=[os.path.join(path,f) for f in os.listdir(path) if f.endswith('SAFE')]
  if len(safepath[0])>1:
    safepath=safepath[0]
  return(safepath)

def metadata_folder(path):
    
    """obtain the path to the metadata file for sentinel-2 tiles 
        
        path = path of the sentinel-2 '.Safe' product file
        
        Returns: the path to the directory containing the metadata file
        
    """
    
    new_path=os.path.join(path,'GRANULE',os.listdir(os.path.join(path,'GRANULE'))[0],'MTD_TL.xml')
    
    return new_path 

def location_datac(path):
    
    """ parse the xml metadata file and retrieve the utm coordinates of the tile
    
        path: path to the xml metadata file
    
        Returns: utm coordinate of the top left and bottom right corners of the tile
    """
    
    tree = (ET.parse(path)).getroot() #parse the file
    
    x_length = 10*int(tree[1][0][2][0].text) #obtain the x_dim length
    
    y_length = 10*int(tree[1][0][2][1].text) #obtain the y_dim length
    
    upper_left_x = int(tree[1][0][5][0].text) #obtain upper left x coordinate
    
    upper_left_y = int(tree[1][0][5][1].text) #obtain upper left y coordinate
    
    lower_right_x = upper_left_x + x_length #obtain lower right x coordinate
    
    lower_right_y = upper_left_y - y_length #obtain lower right y coordinate
    
    return upper_left_x,upper_left_y,lower_right_x,lower_right_y 

## functions to open the sentinel-2 file

In [ ]:
def ten_path(path):
    
    """obtain the path to the 10 metre resolution sentinel-2 tiles from the 
        overall directory
        
        path = path of the sentinel-2 '.Safe' product file
        
        Returns: the path to the directory containing the 10 metre resolution .jp2 files
        
        """
    
    new_path=os.path.join(path,'GRANULE',os.listdir(os.path.join(path,'GRANULE'))[0],'IMG_DATA','R10m')
    
    return new_path

def norm(band):
    band_min, band_max = band.min(), band.max()
    return ((band - band_min)/(band_max - band_min))
    
def load_jp2(path):
    
    """from the path obtained using ten_path() load the .jp2 files and then
       create a 3 channel numpy array, corresponding to a rgb image
        
        path = path of 10m resolution product
        
        Returns: 3 channel array corresponding to rgb tile image
        
        """
    files = sorted(os.listdir(path))
    
    blue = rasterio.open(os.path.join(path,files[1]), driver='JP2OpenJPEG').read(1) #blue band .jp2 file
    green = rasterio.open(os.path.join(path,files[2]), driver='JP2OpenJPEG').read(1)#green band .jp2 file
    red = rasterio.open(os.path.join(path,files[3]), driver='JP2OpenJPEG').read(1) #red band .jp2 file

    image = np.dstack((red,green,blue)) #stack the files and create rgb image

    p2, p98 = np.percentile(image, (2,98)) # rescaling image
    image=exposure.rescale_intensity(image, in_range=(p2, p98))/10000
    if image.max()>1:
      image=image/image.max()
    return image
    
def save_tif(array,filename,load_tif=True):
    
    """save the .jp2 arrays as tifs 
        
        array = array obtained using load_jp2()
        
        filename = 'filename.tif'
        
        load_tif = True ---reload the tif just saved and return it
        load_tif = False --- Don't load or return the tif
        
        Returns: 3 channel array corresponding to rgb tile image
        
        """
    
    tif.imsave(filename,array) #save the array as a tif file
    
    #reload the tif previously saved and return as numpy array
    if load_tif==True:
            
        image_tif = tif.imread(filename)
            
    return image_tif

# Main

## Get the prediction filename and the projection

In [ ]:
projection,predictionfn=directorystructure(dirfiles,primaryorsecondary)

In [ ]:
ds=gdal.Open(predictionfn)
opt=gdal.TranslateOptions(xRes=10, yRes=10) 
gdal.Translate('Prediction10m.tif',ds,options=opt)
ds=None

## Get the sentinel-2 .SAFE directory path

In [ ]:
s2path=sentinel2safepath(dirfiles)

# Get the sentinel-2 metadata path

In [ ]:
metadatapath=metadata_folder(s2path)

## Get the boundary of the sentinel-2 image (optional)

In [ ]:
(ulx,uly,lrx,lry)=location_datac(metadatapath)

## Get the 3 RGB sentinel-2 bands path

In [ ]:
ten_pathi = ten_path(s2path)

## Load the RGB bands normalize them and return them as an array

In [ ]:
image=load_jp2(ten_pathi)

In [ ]:
image=(image[30:10950,30:10950,:]*255).astype(int) #the UW map is cropped because there is an overlay between the sentinel-2 files

## Save the array as a sentinel-2 tif file

In [ ]:
refds=gdal.Open('Prediction10m.tif')
refdriver=refds.GetDriver()

In [ ]:
output_raster=refdriver.Create('rgb.tif',image.shape[0],
                               image.shape[1],
                               image.shape[2],
                               gdal.GDT_Int16)
gt=refds.GetGeoTransform()
out_proj=refds.GetProjection()
output_raster.SetProjection(out_proj)
output_raster.SetGeoTransform(gt)
output_raster.GetRasterBand(1).WriteArray(image[:,:,0])   # write r-band to the raster
output_raster.GetRasterBand(2).WriteArray(image[:,:,1])   # write g-band to the raster
output_raster.GetRasterBand(3).WriteArray(image[:,:,2])   # write b-band to the raster
output_raster.FlushCache() 
output_raster=None

# Plot the 2 files

In [ ]:
fig, axs = plt.subplots(1, 2)
ax0=axs[0]
ax0.imshow(gdal.Open('rgb.tif').ReadAsArray().transpose(1,2,0)[0:1000,0:1000,:])
ax1=axs[1]
ax1.imshow(gdal.Open('Prediction10m.tif').ReadAsArray()[0:1000,0:1000])